In [37]:
from unsloth import FastLanguageModel
from unsloth import UnslothTrainer, UnslothTrainingArguments
from trl import SFTTrainer, SFTConfig
import torch

SEED = 42

In [38]:
model_name = "meta-llama/Llama-3.2-1B-instruct"
MAX_LENGTH = 256

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = MAX_LENGTH,
    full_finetuning=False,
    load_in_4bit = True,
    load_in_8bit = False,
)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
RANK = 512
model = FastLanguageModel.get_peft_model(
    model,
    r = RANK,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = RANK*2,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = SEED,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)
model.print_trainable_parameters()

==((====))==  Unsloth 2025.9.8: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.10.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
trainable params: 360,710,144 || all params: 1,596,524,544 || trainable%: 22.5935


## Datasets

### Simple dataset loading example

In [39]:
import pandas as pd
from langchain.schema import Document
dataset_knowledge = pd.read_csv("../notebooks/data/contacts_docs.csv")
documents = []
for index, row in dataset_knowledge.iterrows():
    doc = f"Nombre: {row['name']}\nTeléfono: {row['phone']}"
    documents.append(Document(page_content=doc, metadata={"id": f"{row['id']}" } ))
print(f"Loaded {len(documents)} documents.")
print(f"First document: {documents[0]}")


Loaded 400 documents.
First document: page_content='Nombre: Alba Alonso
Teléfono: 632 322 183' metadata={'id': '7500_1'}


In [40]:
query_dataset_train = pd.read_csv("../notebooks/data/contacts_queries_train.csv")
query_dataset_val = pd.read_csv("../notebooks/data/contacts_queries_val.csv")
query_dataset_test = pd.read_csv("../notebooks/data/contacts_queries_test.csv")


In [41]:
all_data = {
    "train": query_dataset_train,
    "validation": query_dataset_val,
    "test": query_dataset_test,
}

#to hugginface dataset
from datasets import Dataset, DatasetDict
dataset_qa = {}
for split in all_data:
    dataset_qa[split] = Dataset.from_pandas(all_data[split])
dataset_qa = DatasetDict(dataset_qa)

In [42]:
dataset_qa

DatasetDict({
    train: Dataset({
        features: ['question', 'id', 'respuesta'],
        num_rows: 1400
    })
    validation: Dataset({
        features: ['question', 'id', 'respuesta'],
        num_rows: 300
    })
    test: Dataset({
        features: ['question', 'id', 'respuesta'],
        num_rows: 300
    })
})

In [43]:
# JOIN TRAIN AND VAL DATASETS
from datasets import concatenate_datasets
dataset_qa["train"] = concatenate_datasets([dataset_qa["train"], dataset_qa["validation"]])

In [44]:
# rename "respuesta" column to "answer"
dataset_qa = dataset_qa.rename_column("respuesta", "answer")

In [45]:
dataset_qa

DatasetDict({
    train: Dataset({
        features: ['question', 'id', 'answer'],
        num_rows: 1700
    })
    validation: Dataset({
        features: ['question', 'id', 'answer'],
        num_rows: 300
    })
    test: Dataset({
        features: ['question', 'id', 'answer'],
        num_rows: 300
    })
})

### Data preparation

In [46]:
def build_prompt_it(tokenizer, system_prompt: str, prompt: str, response: str) -> str:
    """Builds the chat prompt for a single example using the tokenizer chat template."""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": prompt},
        {"role": "assistant", "content": response}
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
    )

In [47]:
def generate_knowledge_injection_prompts(documents: list):
    prompt = """{doc}"""
    for doc in documents:
        yield prompt.format(doc=doc.page_content)

In [48]:
def generate_qa_prompts(dataset, tokenizer):
    system_prompt = """
    Eres un modelo de lenguaje entrenado para responder preguntas.
    """
    prompts = []
    for item in dataset:
        prompt = """{QUERY}"""
        response = "{response}"
        question = item["question"]
        prompt = prompt.format(QUERY=question)
        prompts.append(build_prompt_it(tokenizer, system_prompt, prompt, response.format(response=item["answer"])))
    return prompts

In [49]:
prompts = list(generate_knowledge_injection_prompts(documents))
print(f"Number of prompts: {len(prompts)}")

Number of prompts: 400


In [50]:
prompts[0]

'Nombre: Alba Alonso\nTeléfono: 632 322 183'

In [51]:
# QUIERO VER LOS TOKENS
def print_tokens(text):
    tokens = tokenizer.tokenize(text)
    print("Number of tokens:", len(tokens), "\n")
    print("Tokens:", tokens)

print_tokens(prompts[0])

Number of tokens: 15 

Tokens: ['Nombre', ':', 'ĠAl', 'ba', 'ĠAlonso', 'Ċ', 'Tel', 'Ã©fono', ':', 'Ġ', '632', 'Ġ', '322', 'Ġ', '183']


In [52]:
# create dataset from prompts
from datasets import Dataset
knowledge_dataset = Dataset.from_dict({"text": prompts})
knowledge_dataset

Dataset({
    features: ['text'],
    num_rows: 400
})

In [53]:
knowledge_dataset["text"][0]

'Nombre: Alba Alonso\nTeléfono: 632 322 183'

In [54]:
prompts_qa_train = generate_qa_prompts(dataset_qa["train"], tokenizer)
prompts_qa_val = generate_qa_prompts(dataset_qa["test"], tokenizer)

print(f"Number of retrieval prompts: {len(prompts_qa_train)}")
print(f"Number of retrieval prompts: {len(prompts_qa_val)}")

Number of retrieval prompts: 1700
Number of retrieval prompts: 300


In [55]:
print(prompts_qa_train[0], sep="\n")

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Nov 2025

Eres un modelo de lenguaje entrenado para responder preguntas.<|eot_id|><|start_header_id|>user<|end_header_id|>

Necesito el contacto asociado al 620 152 344. —consulta interna—<|eot_id|><|start_header_id|>assistant<|end_header_id|>

El número 620 152 344 pertenece a Alejandro Vega.<|eot_id|>


In [56]:
print_tokens(prompts_qa_train[0])

Number of tokens: 87 

Tokens: ['<|begin_of_text|>', '<|start_header_id|>', 'system', '<|end_header_id|>', 'ĊĊ', 'Cut', 'ting', 'ĠKnowledge', 'ĠDate', ':', 'ĠDecember', 'Ġ', '202', '3', 'Ċ', 'Today', 'ĠDate', ':', 'Ġ', '07', 'ĠNov', 'Ġ', '202', '5', 'ĊĊ', 'E', 'res', 'Ġun', 'Ġmodelo', 'Ġde', 'Ġl', 'engu', 'aje', 'Ġentren', 'ado', 'Ġpara', 'Ġresponder', 'Ġpreg', 'untas', '.', '<|eot_id|>', '<|start_header_id|>', 'user', '<|end_header_id|>', 'ĊĊ', 'N', 'ec', 'es', 'ito', 'Ġel', 'Ġcontacto', 'Ġasoci', 'ado', 'Ġal', 'Ġ', '620', 'Ġ', '152', 'Ġ', '344', '.', 'ĠâĢĶ', 'consulta', 'Ġintern', 'a', 'âĢĶ', '<|eot_id|>', '<|start_header_id|>', 'assistant', '<|end_header_id|>', 'ĊĊ', 'El', 'ĠnÃºmero', 'Ġ', '620', 'Ġ', '152', 'Ġ', '344', 'Ġpert', 'ene', 'ce', 'Ġa', 'ĠAlejandro', 'ĠVega', '.', '<|eot_id|>']


In [57]:
# create dataset from prompts train, val, test
qa_train_dataset = Dataset.from_dict({"text": prompts_qa_train})
qa_val_dataset = Dataset.from_dict({"text": prompts_qa_val})

qa_dataset = {
    "train": qa_train_dataset,
    "validation": qa_val_dataset,
}

In [58]:
def tokenize_function_autoregressive(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=MAX_LENGTH)

In [59]:
knowledge_dataset_tokenizer = knowledge_dataset.map(tokenize_function_autoregressive, batched=True)

Map: 100%|██████████| 400/400 [00:00<00:00, 12954.68 examples/s]


In [60]:
qa_train_dataset_tokenizer = qa_dataset["train"].map(tokenize_function_autoregressive, batched=True)
qa_val_dataset_tokenizer = qa_dataset["validation"].map(tokenize_function_autoregressive, batched=True)

Map: 100%|██████████| 300/300 [00:00<00:00, 15787.84 examples/s]


In [61]:
def generate_qa_prompts_testing(dataset, tokenizer):
    def build_prompt_it_generation(tokenizer, system_prompt: str, prompt: str) -> str:
        """Builds the chat prompt for a single example using the tokenizer chat template."""
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": prompt},
        ]
        return tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False,
        )
    system_prompt = """
    Eres un modelo de lenguaje entrenado para responder preguntas.
    """
    prompts = []
    for item in dataset:
        prompt = """{QUERY}"""
        question = item["question"]
        answer = item["answer"]
        prompt = prompt.format(QUERY=question)
        prompts.append(
            (
                build_prompt_it_generation(tokenizer, system_prompt, prompt),
                answer,
            )
        )
    return prompts

In [62]:
prompts_retrieval_test = generate_qa_prompts_testing(dataset_qa["test"], tokenizer)
text_for_testing = prompts_retrieval_test[0][0]
print(text_for_testing)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Nov 2025

Eres un modelo de lenguaje entrenado para responder preguntas.<|eot_id|><|start_header_id|>user<|end_header_id|>

¿A quién pertenece el teléfono 736 615 948?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [63]:
import re
import tqdm
import torch

def levenshtein(s: str, t: str) -> int:
    """
    Distancia de Levenshtein con memoria O(min(len(s), len(t))).
    """
    # Asegura que t es la más corta para usar menos memoria
    if len(s) < len(t):
        s, t = t, s
    previous = list(range(len(t) + 1))

    for i, cs in enumerate(s, start=1):
        current = [i]
        for j, ct in enumerate(t, start=1):
            costo = 0 if cs == ct else 1
            current.append(min(
                current[-1] + 1,          # inserción
                previous[j] + 1,          # borrado
                previous[j-1] + costo     # sustitución
            ))
        previous = current
    return previous[-1]


def test_model_accuracy(model, tokenizer, prompts_retrieval_test, device="cuda", 
                        max_new_tokens=64, temperature=0.0, top_p=1.0):
    
    acc = 0
    lev = 0
    total = 0

    progress_bar = tqdm.tqdm(prompts_retrieval_test, desc="Testing")

    for text, answer in progress_bar:
        inputs = tokenizer(text, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                temperature=temperature,
                top_p=top_p
            )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = generated_text.split("assistant")[-1].strip()

        if response == answer:
            acc += 1
        lev += levenshtein(response, answer)
        total += 1

        progress_bar.set_postfix({
            "acc": f"{acc/total*100:.2f} %",
            "lev": f"{lev/total:.2f}"
        })

    final_acc = acc / total * 100 if total > 0 else 0.0
    print(f"Accuracy final: {acc}/{total} = {final_acc:.2f} %")
    final_lev = lev / total if total > 0 else 0.0
    print(f"Levenshtein final: {final_lev:.2f}")
    return final_acc, final_lev

In [64]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    """
    Genera gráficos de la evolución de la accuracy y las pérdidas (SFT e IT) durante el entrenamiento.

    Parámetros:
        history (list[dict]): lista de diccionarios, donde cada elemento debe contener:
            - "super_epoch": número de super-época.
            - "accuracy": precisión alcanzada.
            - "trainer_sft_stats".training_loss: pérdida SFT.
            - "trainer_it_stats".training_loss: pérdida IT.
    """
    # Extraer métricas del historial
    super_epochs = [h["super_epoch"] for h in history]
    accuracies = [h["accuracy"] for h in history]
    losses_sft = [h["trainer_sft_stats"].training_loss for h in history]
    losses_it = [h["trainer_it_stats"].training_loss for h in history]

    # --- Gráfico de Accuracy ---
    plt.figure(figsize=(6, 4))
    plt.plot(super_epochs, accuracies, marker='o', color='tab:blue')
    plt.xlabel("Super Epoch")
    plt.ylabel("Accuracy (%)")
    plt.title("Accuracy History")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # --- Gráfico de Losses ---
    plt.figure(figsize=(6, 4))
    plt.plot(super_epochs, losses_sft, marker='o', label="SFT Loss", color='tab:orange')
    plt.plot(super_epochs, losses_it, marker='s', label="IT Loss", color='tab:green')
    plt.xlabel("Super Epoch")
    plt.ylabel("Loss")
    plt.title("Loss History")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


## Train

### Iterative training configuration

In [65]:
# sft training
from transformers import DataCollatorForLanguageModeling


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
auto_config = UnslothTrainingArguments(
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 8, # Use GA to mimic batch size!
    save_strategy="no",
    save_total_limit=0,
    warmup_steps = 5,
    num_train_epochs = 1, # Set this for 1 full training run.
    #max_steps = 60,
    learning_rate = 1e-4, # Reduce to 2e-5 for long training runs
    logging_steps = 1,
    # 32 bits
    optim = "paged_adamw_32bit",
    weight_decay = 0.01,
    lr_scheduler_type = "cosine",
    seed = SEED,
    report_to = "none", # Use this for WandB etc
    output_dir="../models/qwen3-0.6b-rag-indexer",
)

it_config = SFTConfig(
    dataset_text_field="text",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,        # <-- añade eval batch size
    gradient_accumulation_steps=8,
    warmup_steps=25,
    save_strategy="no",
    save_total_limit=0,
    eval_steps=1,
    eval_strategy="steps",         # <-- activa evaluación periódica
    num_train_epochs=1,             # <-- opcional: usa epochs en lugar de max_steps
    #max_steps=30,
    learning_rate=1e-4,
    logging_steps=1,
    optim = "paged_adamw_32bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=SEED,
    report_to="none",
    output_dir="../models/qwen3-0.6b-rag-retriever",
    load_best_model_at_end=False,          # <-- opcional
    metric_for_best_model="eval_loss",    # <-- opcional
    greater_is_better=False,              # <-- opcional
)

trainer_auto = UnslothTrainer(
    model=model,
    train_dataset=knowledge_dataset_tokenizer,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=auto_config,
)

trainer_it = SFTTrainer(
    model=model,
    train_dataset=qa_train_dataset_tokenizer,
    eval_dataset=qa_val_dataset_tokenizer,
    data_collator=data_collator,
    tokenizer=tokenizer,
    args=it_config,
)

In [66]:
model.print_trainable_parameters()

trainable params: 360,710,144 || all params: 1,596,524,544 || trainable%: 22.5935


In [67]:
import wandb

name = f"{model_name.replace('/', '_')}_r{RANK}_qa_iterativo_agenda"

# Inicia la sesión de wandb
wandb.init(
    project="qa_iterativo_agenda",
    name=name,  # opcional
    config={
        "super_epochs": 10,
        "model": model_name,
        "r": RANK,
    },
)

wandb: Currently logged in as: miguel_kjh to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [68]:
EPOCHS = 10

for _ in range(EPOCHS):
    print(f"--- SUPER EPOCH {_+1} / {EPOCHS} ---")
    trainer_sft_stats = trainer_auto.train() 
    trainer_it_stats = trainer_it.train()
    # evaluate accuracy after each super epoch
    acc, lev = test_model_accuracy(model, tokenizer, prompts_retrieval_test, device="cuda")
    wandb.log({
        "super_epoch": _ + 1,
        "accuracy": acc,
        "levenshtein": lev,
        "train_loss_sft": trainer_sft_stats.training_loss,
        "train_loss_it": trainer_it_stats.training_loss,
    })

wandb.finish()

--- SUPER EPOCH 1 / 10 ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 400 | Num Epochs = 1 | Total steps = 7
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 360,710,144 of 1,596,524,544 (22.59% trained)


Step,Training Loss
1,4.456800
2,4.408200
3,3.169100
4,2.797400
5,2.203600
6,1.899700
7,1.934600


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,700 | Num Epochs = 1 | Total steps = 27
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 360,710,144 of 1,596,524,544 (22.59% trained)


Step,Training Loss,Validation Loss
1,6.493000,6.481918
2,6.432200,5.049185
3,5.053000,3.767687
4,3.729300,2.774565
5,2.753400,2.141159
6,2.138400,1.656620
7,1.621900,1.270533
8,1.258000,0.923814
9,0.961500,0.790050
10,0.813000,0.770230


...

Step,Training Loss,Validation Loss
1,0.239000,0.256639
2,0.235900,0.254685
3,0.235800,0.251143
4,0.230400,0.242479
5,0.219400,0.232565
6,0.207300,0.218279
7,0.196000,0.202223
8,0.181400,0.184405
9,0.163000,0.171183
10,0.152000,0.162905


Testing: 100%|██████████| 300/300 [01:44<00:00,  2.86it/s, acc=95.33 %, lev=0.50]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Accuracy final: 286/300 = 95.33 %
Levenshtein final: 0.50


wandb: ERROR Control-C detected -- Run data was not synced


KeyboardInterrupt: 

In [ ]:
# test the model in streaming mode
from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=False)
_ = model.generate(
    **tokenizer(text_for_testing, return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    do_sample = False,
    top_p = 0.1,
    temperature = 0.,
    streamer = streamer,
)

El número 736 615 948 pertenece a Leo Pérez.<|eot_id|>


In [ ]:
acc, lev = test_model_accuracy(model, tokenizer, prompts_retrieval_test, device="cuda")

Testing: 100%|██████████| 300/300 [01:45<00:00,  2.84it/s, acc=91.00 %, lev=0.80]

Accuracy final: 273/300 = 91.00 %
Levenshtein final: 241/300 = 0.80
